<a href="https://colab.research.google.com/github/nescoba/portafolio/blob/main/backfitting_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install itscalledsoccer

     |████████████████████████████████| 61 kB 1.9 MB/s 
     |████████████████████████████████| 11.5 MB 6.0 MB/s 
     |████████████████████████████████| 893 kB 46.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: CacheControl
    Found existing installation: CacheControl 0.12.10
    Uninstalling CacheControl-0.12.10:
      Successfully uninstalled CacheControl-0.12.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.25.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have fo

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 
sns.set()

In [3]:
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from itscalledsoccer.client import AmericanSoccerAnalysis

asa_client = AmericanSoccerAnalysis() 

Gathering all players
Gathering all teams
Gathering all stadia
Gathering all managers
Gathering all referees
Finished initializing client


In [5]:
def construct_time_series_date(team_id, date):
    seasons = asa_client.get_game_xgoals(leagues='mls')
    seasons.loc[:, 'date_formated'] = pd.to_datetime(seasons['date_time_utc'])
    seasons = seasons[seasons['date_formated'] < date]

    seasons_team = seasons[(seasons['home_team_id']==team_id) | (seasons['away_team_id']==team_id)]
    team_home = seasons_team[seasons_team['home_team_id']==team_id]
    team_away = seasons_team[seasons_team['away_team_id']==team_id]

    team_home.loc[:,'teamxgoals'] = team_home['home_team_xgoals']
    team_away.loc[:,'teamxgoals'] = team_away['away_team_xgoals']

    team_home.loc[:,'teamgoals'] = team_home['home_goals']
    team_away.loc[:,'teamgoals'] = team_away['away_goals']

    team_home.loc[:,'oppoxgoals'] = team_home['away_team_xgoals']
    team_away.loc[:,'oppoxgoals'] = team_away['home_team_xgoals']

    team_home.loc[:,'oppogoals'] = team_home['away_goals']
    team_away.loc[:,'oppogoals'] = team_away['home_goals']

    team_home_red = team_home.loc[:, ['teamgoals', 'teamxgoals', 'oppoxgoals', 'oppogoals', 'date_formated']]
    team_away_red = team_away.loc[:, ['teamgoals', 'teamxgoals', 'oppoxgoals', 'oppogoals', 'date_formated']]

    team_red = pd.merge(team_home_red, team_away_red, how='outer')

    team_red = team_red.sort_values(by='date_formated')

    return team_red

In [6]:
def normal_with_nans(mean, sd):
    try:
        return np.random.normal(mean, sd)
    except:
        return np.nan

def poisson_with_nans(mean):
    try:
        return np.random.poisson(mean)
    except:
        return 0

In [7]:
def probs_game(homeid, awayid, date):

    n_simulations = 1000
    n_home = 0
    n_draw = 0
    n_away = 0

    

    homedf = construct_time_series_date(homeid, date)
    awaydf = construct_time_series_date(awayid, date)



    homedf.loc[:, 'rollxgmean'] = homedf['teamxgoals'].rolling(5).mean()
    homedf.loc[:, 'rollxgstd'] = homedf['teamxgoals'].rolling(5).std()

    awaydf.loc[:, 'rolloppoxgmean'] = awaydf['oppoxgoals'].rolling(5).mean()
    awaydf.loc[:, 'rolloppoxgstd'] = awaydf['oppoxgoals'].rolling(5).std()

    awaydf.loc[:, 'rollxgmean'] = awaydf['teamxgoals'].rolling(5).mean()
    awaydf.loc[:, 'rollxgstd'] = awaydf['teamxgoals'].rolling(5).std()

    homedf.loc[:, 'rolloppoxgmean'] = homedf['oppoxgoals'].rolling(5).mean()
    homedf.loc[:, 'rolloppoxgstd'] = homedf['oppoxgoals'].rolling(5).std()


    for n in range(n_simulations):

        try:
            homepredxgoals = normal_with_nans(homedf['rollxgmean'].iloc[-1], homedf['rollxgstd'].iloc[-1])
        except:
            homepredxgoals = 1


        try:
            awaypredoppoxgoals = normal_with_nans(awaydf['rolloppoxgmean'].iloc[-1], awaydf['rolloppoxgstd'].iloc[-1])
        except:
            awaypredoppoxgoals=1

        hometoawaypredxgoals = np.mean(np.array([float(homepredxgoals),float(awaypredoppoxgoals)]))

        
        try:
            awaypredxgoals = normal_with_nans(awaydf['rollxgmean'].iloc[-1], awaydf['rollxgstd'].iloc[-1])
        except:
            awaypredxgoals = 1
                   

        try:
            homepredoppoxgoals = normal_with_nans(homedf['rolloppoxgmean'].iloc[-1], homedf['rolloppoxgstd'].iloc[-1])
        except:
            homepredoppoxgoals = 1

        awaytohomepredxgoals = np.mean(np.array([float(awaypredxgoals), float(homepredoppoxgoals)]))
    
        home_score = poisson_with_nans(hometoawaypredxgoals)
        away_score = poisson_with_nans(awaytohomepredxgoals)
        if home_score > away_score:
            n_home += 1
        elif home_score < away_score:
            n_away += 1
        else:
            n_draw += 1 

    return (n_home / n_simulations, n_draw / n_simulations, n_away / n_simulations)

In [8]:
def probs_game_from_id(matchid):
    all_games = asa_client.get_games(leagues='mls')
    homeid = all_games.loc[all_games['game_id']==matchid, 'home_team_id'].values[0]

    awayid = all_games.loc[all_games['game_id']==matchid, 'away_team_id'].values[0]

    date = pd.to_datetime(all_games.loc[all_games['game_id']==matchid, 'date_time_utc'].values[0])

    print(date)

    probs_game(homeid, awayid, date)


In [9]:
def choose_result_from_id(matchid):
    return np.random.choice(['home', 'draw', 'away'], p=probs_game_from_id(matchid))

In [10]:
choose_result_from_id('9z5knLXjMA')

2022-03-06 21:00:00+00:00


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


'away'

In [11]:
v_choose_result_from_id = np.vectorize(choose_result_from_id)

In [12]:
all_games = asa_client.get_games(leagues='mls')

games2021 = all_games.iloc[:200, :]

games2021

,game_id,date_time_utc,home_score,away_score,home_team_id,away_team_id,referee_id,stadium_id,home_manager_id,away_manager_id,expanded_minutes,season_name,matchday,attendance,knockout_game,last_updated_utc,extra_time,penalties,home_penalties,away_penalties
0,315VVxG959,2022-03-07 03:00:00 UTC,1,1,eVq3ya6MWO,WBLMvYAQxe,e7MzPKXqr0,7vQ7xbOMD1,NWMWxKe5lz,gpMOYv1qzy,100,2022,2,22102,False,2022-03-07 05:30:09 UTC,NaN,NaN,NaN,NaN
1,9z5knLXjMA,2022-03-06 21:00:00 UTC,5,1,gpMOLwl5zy,zeQZkL1MKw,jYQJnoVQGR,jYQJZDDQGR,0Oq6mNP56D,gpMOYJ7qzy,100,2022,2,20738,False,2022-03-06 23:38:24 UTC,NaN,NaN,NaN,NaN
2,KAqBD3KBMb,2022-03-06 00:30:00 UTC,0,1,NPqxKXZ59d,kaDQ0wRqEv,9z5kJ7gMA3,KXMeOR2q64,gjMNG4k5Kp,Xj5Yx60Mbd,100,2022,2,74479,False,2022-03-07 03:55:19 UTC,NaN,NaN,NaN,NaN
3,e7Mz8VGGqr,2022-03-05 23:00:00 UTC,1,1,kRQand1MKZ,vzqoOgNqap,gjMNk4v5Kp,odMXGRrQYL,OlMlYvy5Lz,gOMnDkAqwN,96,2022,2,0,False,2022-03-07 01:57:19 UTC,NaN,NaN,NaN,NaN
4,eVq3gvKVqW,2022-03-05 23:00:00 UTC,1,0,a2lqR4JMr0,jYQJ19EqGR,7VqG6KX5vW,e7MzlRjqr0,4wM4vOAqjB,odMXxreMYL,95,2022,2,20017,False,2022-03-06 03:22:17 UTC,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,KXMeYWJ3M6,2021-09-12 00:00:00 UTC,0,1,APk5LGOMOW,vzqoOgNqap,zeQZl7D5Kw,NWMW8pj5lz,KPqjO38Q6v,gOMnDkAqwN,96,2021,24,0,False,2021-09-13 11:44:27 UTC,NaN,NaN,NaN,NaN
196,OlMlaR6A5L,2021-09-12 00:00:00 UTC,2,0,NWMWlBK5lz,kRQabn8MKZ,gOMnB7AMwN,Oa5wKLY514,eV5DWdEMKn,7VqGPRYQvW,100,2021,24,22636,False,2021-09-15 06:34:58 UTC,NaN,NaN,NaN,NaN
197,2vQ1lr2Lqr,2021-09-11 23:00:00 UTC,1,1,a2lqRX2Mr0,EKXMeX3Q64,KAqB3m7Qbg,NWMW84L5lz,9z5kKNb5A3,e7MzE8XQr0,96,2021,24,12574,False,2021-09-16 13:12:36 UTC,NaN,NaN,NaN,NaN
198,Pk5LJVvPQO,2021-09-11 23:00:00 UTC,2,1,19vQ2095K6,Vj58weDM8n,jYQJnoVQGR,wvq9p775Wn,LeVq3j5WOJ,2lqRa4D5r0,98,2021,24,19481,False,2021-09-15 07:02:39 UTC,NaN,NaN,NaN,NaN


In [14]:
games2021.loc[:, 'projected_result'] = v_choose_result_from_id(games2021['game_id'])

games2021

2022-03-07 03:00:00+00:00


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


2022-03-07 03:00:00+00:00
2022-03-06 21:00:00+00:00
2022-03-06 00:30:00+00:00
2022-03-05 23:00:00+00:00
2022-03-05 23:00:00+00:00
2022-03-05 23:00:00+00:00
2022-03-05 23:00:00+00:00
2022-03-05 23:00:00+00:00
2022-03-05 23:00:00+00:00
2022-03-05 22:30:00+00:00
2022-03-05 21:00:00+00:00
2022-03-05 20:30:00+00:00
2022-03-05 19:00:00+00:00
2022-03-05 18:30:00+00:00
2022-02-28 01:00:00+00:00
2022-02-28 00:00:00+00:00
2022-02-27 22:00:00+00:00
2022-02-27 20:00:00+00:00
2022-02-27 18:00:00+00:00
2022-02-27 00:30:00+00:00
2022-02-26 23:00:00+00:00
2022-02-26 23:00:00+00:00
2022-02-26 23:00:00+00:00
2022-02-26 23:00:00+00:00
2022-02-26 22:30:00+00:00
2022-02-26 20:30:00+00:00
2022-02-26 20:30:00+00:00
2022-02-26 18:00:00+00:00
2021-12-11 20:00:00+00:00
2021-12-05 20:00:00+00:00
2021-12-04 23:30:00+00:00
2021-12-01 00:30:00+00:00
2021-11-28 22:30:00+00:00
2021-11-28 20:00:00+00:00
2021-11-25 21:30:00+00:00
2021-11-24 03:30:00+00:00
2021-11-24 01:00:00+00:00
2021-11-21 22:30:00+00:00
2021-11-21 2

,game_id,date_time_utc,home_score,away_score,home_team_id,away_team_id,referee_id,stadium_id,home_manager_id,away_manager_id,...,season_name,matchday,attendance,knockout_game,last_updated_utc,extra_time,penalties,home_penalties,away_penalties,projected_result
0,315VVxG959,2022-03-07 03:00:00 UTC,1,1,eVq3ya6MWO,WBLMvYAQxe,e7MzPKXqr0,7vQ7xbOMD1,NWMWxKe5lz,gpMOYv1qzy,...,2022,2,22102,False,2022-03-07 05:30:09 UTC,NaN,NaN,NaN,NaN,draw
1,9z5knLXjMA,2022-03-06 21:00:00 UTC,5,1,gpMOLwl5zy,zeQZkL1MKw,jYQJnoVQGR,jYQJZDDQGR,0Oq6mNP56D,gpMOYJ7qzy,...,2022,2,20738,False,2022-03-06 23:38:24 UTC,NaN,NaN,NaN,NaN,away
2,KAqBD3KBMb,2022-03-06 00:30:00 UTC,0,1,NPqxKXZ59d,kaDQ0wRqEv,9z5kJ7gMA3,KXMeOR2q64,gjMNG4k5Kp,Xj5Yx60Mbd,...,2022,2,74479,False,2022-03-07 03:55:19 UTC,NaN,NaN,NaN,NaN,draw
3,e7Mz8VGGqr,2022-03-05 23:00:00 UTC,1,1,kRQand1MKZ,vzqoOgNqap,gjMNk4v5Kp,odMXGRrQYL,OlMlYvy5Lz,gOMnDkAqwN,...,2022,2,0,False,2022-03-07 01:57:19 UTC,NaN,NaN,NaN,NaN,draw
4,eVq3gvKVqW,2022-03-05 23:00:00 UTC,1,0,a2lqR4JMr0,jYQJ19EqGR,7VqG6KX5vW,e7MzlRjqr0,4wM4vOAqjB,odMXxreMYL,...,2022,2,20017,False,2022-03-06 03:22:17 UTC,NaN,NaN,NaN,NaN,home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,KXMeYWJ3M6,2021-09-12 00:00:00 UTC,0,1,APk5LGOMOW,vzqoOgNqap,zeQZl7D5Kw,NWMW8pj5lz,KPqjO38Q6v,gOMnDkAqwN,...,2021,24,0,False,2021-09-13 11:44:27 UTC,NaN,NaN,NaN,NaN,draw
196,OlMlaR6A5L,2021-09-12 00:00:00 UTC,2,0,NWMWlBK5lz,kRQabn8MKZ,gOMnB7AMwN,Oa5wKLY514,eV5DWdEMKn,7VqGPRYQvW,...,2021,24,22636,False,2021-09-15 06:34:58 UTC,NaN,NaN,NaN,NaN,home
197,2vQ1lr2Lqr,2021-09-11 23:00:00 UTC,1,1,a2lqRX2Mr0,EKXMeX3Q64,KAqB3m7Qbg,NWMW84L5lz,9z5kKNb5A3,e7MzE8XQr0,...,2021,24,12574,False,2021-09-16 13:12:36 UTC,NaN,NaN,NaN,NaN,home
198,Pk5LJVvPQO,2021-09-11 23:00:00 UTC,2,1,19vQ2095K6,Vj58weDM8n,jYQJnoVQGR,wvq9p775Wn,LeVq3j5WOJ,2lqRa4D5r0,...,2021,24,19481,False,2021-09-15 07:02:39 UTC,NaN,NaN,NaN,NaN,home


In [18]:
def decide_result(game_id):
    games = asa_client.get_games(leagues='mls')
    game = games[games['game_id']==game_id]
    home_score = game['home_score'].values[0]
    away_score = game['away_score'].values[0]

    if home_score > away_score:
        return 'home'
    elif away_score > home_score:
        return 'away'
    else:
        return 'draw'

In [19]:
v_decide_result = np.vectorize(decide_result)

In [20]:
games2021.loc[:, 'result'] = v_decide_result(games2021['game_id'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(games2021['result'], games2021['projected_result'])

0.29

In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix(games2021['result'], games2021['projected_result'])

array([[16, 17, 17],
       [18, 17, 18],
       [39, 33, 25]])